<a href="https://colab.research.google.com/github/ezinneanne/DeepTechReady-Work/blob/deeptech/Week2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment: Build and Evaluate Multiclass CNN Models for Fish or Boat Dataset

## Using Boat Dataset

In [ ]:
# Mounting Gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# UNZIP DATASET
import zipfile, os

# Define the path to the zipped dataset and where to extract it
zip_path = '/content/drive/MyDrive/Boat dataset(Assignment).zip'
extract_path = '/content/boat_dataset'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the dataset to the extraction path
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped.")

Dataset unzipped.


In [ ]:
import tensorflow as tf #importing tensorflow for deep learning functionality
from tensorflow.keras import layers, models # Import the 'layers' and 'models' submodules from tensorflow.keras
# 'layers' is used to build different types of neural network layers (e.g., Conv2D, Dense, etc.)
# 'models' provides APIs to create and manage models (Sequential and Functional APIs)

from tensorflow.keras.applications import MobileNetV2 # Import the pre-trained MobileNetV2 model from keras.applications
# MobileNetV2 is a lightweight deep convolutional neural network architecture for mobile and edge devices
# It can be used as a feature extractor or a full model for transfer learning

from sklearn.metrics import classification_report # to make classification report for evaluation
import numpy as np # importing numpy for numerical operations

from tensorflow.keras.models import Sequential  # Importing Sequential model for linear stacking of layers

In [ ]:
# Defining parameters for image processing
batch_size = 32  # Set batch size for loading data (number of samples per batch)
img_height = 224  # Set height for resizing input images
img_width = 224  # Set width for resizing input images
data_path = os.path.join(extract_path, 'boat data', 'test') #defining path for the data to be used

In [ ]:
# LOAD DATASET AND SPLIT
# Load training data from the dataset directory, with 80% for training
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,                 # Path to images
    validation_split=0.2,     # 20% for validation
    subset="training",        # This is the training subset
    seed=123,                 # Seed for consistent split
    image_size=(img_height, img_width),  # Resize all images
    batch_size=batch_size     # Number of images per batch
)

# Load validation data (remaining 20%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Get class names and number of classes
class_names = train_ds.class_names
num_classes = len(class_names)

Found 4774 files belonging to 24 classes.
Using 3820 files for training.
Found 4774 files belonging to 24 classes.
Using 954 files for validation.


In [ ]:
# PREFETCHING FOR PERFORMANCE
AUTOTUNE = tf.data.AUTOTUNE

# Cache, shuffle, and prefetch training dataset for better performance
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

# Cache and prefetch validation dataset (no shuffling needed)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# CUSTOM CNN MODEL
custom_model = models.Sequential([

    #first convolutional block
    layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    # 32 filters, 3x3 kernel, ReLU activation, input shape for RGB image

    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    # Second conv layer with 64 filters, keeps dimensions the same due to 'same' padding

    layers.MaxPooling2D(), # reduce spatial size
    # Downsamples feature maps using 2x2 pool size (default)

    layers.Dropout(0.25), #prevent overfitting
    # Randomly drops 25% of the nodes to reduce overfitting


    #second convolutional block
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    # Another conv layer with 64 filters

    layers.MaxPooling2D(),
    # Downsampling again

    layers.Dropout(0.25),
    #Dropout to regularize

    #third convolutional block
    layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    # Deeper conv layer with 128 filters

    layers.MaxPooling2D(), # Downsampling again
    layers.Dropout(0.25), # Dropout to regularize p

    # fully connected(dense) layers
    layers.Flatten(), # Flatten 3D feature maps into 1D vector for dense layers
    layers.Dense(128, activation='relu'),  # Dense layer with 128 neurons
    layers.Dropout(0.5), # Dropout of 50% to further prevent overfitting
    layers.Dense(num_classes, activation='softmax')  # Output layer with the number of classes in the dataset(for multi classification), sigmoid activation for probability output

])

# compile the custom model with optimizer, loss, and evaluation metric for accuracy
custom_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy', #categorical loss for multi classification
                     metrics=['accuracy']) #tracking accuracy for training and validation

#Train the model
print("Training Custom CNN...")
custom_model.fit(train_ds, validation_data=val_ds, epochs=5)

Training Custom CNN...
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


120/120 ━━━━━━━━━━━━━━━━━━━━ 1323s 11s/step - accuracy: 0.1538 - loss: 120.6771 - val_accuracy: 0.4717 - val_loss: 2.2492
Epoch 2/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 1341s 11s/step - accuracy: 0.4281 - loss: 2.0852 - val_accuracy: 0.5996 - val_loss: 1.4487
Epoch 3/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 1339s 11s/step - accuracy: 0.5309 - loss: 1.7074 - val_accuracy: 0.6562 - val_loss: 1.3444
Epoch 4/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 1317s 11s/step - accuracy: 0.5848 - loss: 1.4917 - val_accuracy: 0.6667 - val_loss: 1.2463
Epoch 5/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 1317s 11s/step - accuracy: 0.6091 - loss: 1.3685 - val_accuracy: 0.6635 - val_loss: 1.1692


In [ ]:
custom_model.summary() # show the model architecture

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 224, 224, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,935,370 (148.53 MB)

 Trainable params: 12,978,456 (49.51 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,956,914 (99.02 MB)

In [ ]:
# Defining parameters for image processing
batch_size = 32  # Set batch size for loading data (number of samples per batch)
image_height = 224  # Set height for resizing input images
image_width = 224  # Set width for resizing input images

In [ ]:
# LOAD DATASET AND SPLIT
# Load training data from the dataset directory, with 80% for training
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,                 # Path to images
    validation_split=0.2,     # 20% for validation
    subset="training",        # This is the training subset
    seed=123,                 # Seed for consistent split
    image_size=(image_height, image_width),  # Resize all images
    batch_size=batch_size     # Number of images per batch
)

# Load validation data (remaining 20%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

# Get class names and number of classes
class_names = train_ds.class_names
num_classes = len(class_names)

Found 4774 files belonging to 24 classes.
Using 3820 files for training.
Found 4774 files belonging to 24 classes.
Using 954 files for validation.


In [ ]:
# TRANSFER LEARNING WITH MOBILENETV2
# Load MobileNetV2 without top layer and with pretrained weights
# Load the MobileNetV2 model with pre-trained weights from ImageNet
# weights='imagenet' means the model is loaded with weights learned from training on the ImageNet dataset
# input_shape specifies the shape of input images (height, width, 3 channels for RGB)
# include_top=False excludes the fully connected layers at the top of the model (used for classification in ImageNet)
# This allows you to add your own custom classification layers on top (for transfer learning)

base_model = MobileNetV2(input_shape=(image_height, image_width, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # Freeze base model layers so its weights will not be updated during training

# Build model on top of MobileNetV2
mobilenet_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),       # Reduce dimensions
    layers.Dense(128, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),                   # Dropout layer
    layers.Dense(num_classes, activation='softmax')  # Output layer
])



mobilenet_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

print("Training MobileNetV2...")
mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=5)

Training MobileNetV2...
Epoch 1/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.4560 - loss: 2.0319 - val_accuracy: 0.6279 - val_loss: 1.2289
Epoch 2/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.5841 - loss: 1.3967 - val_accuracy: 0.6950 - val_loss: 1.0908
Epoch 3/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.6186 - loss: 1.2513 - val_accuracy: 0.6971 - val_loss: 1.0604
Epoch 4/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.6621 - loss: 1.1576 - val_accuracy: 0.7285 - val_loss: 0.9377
Epoch 5/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.6754 - loss: 1.0825 - val_accuracy: 0.7296 - val_loss: 0.9124


In [ ]:
def evaluate_model(model, dataset, name="Model"):
    y_true = []
    y_pred = []

    for images, labels in dataset:
        preds = model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Get unique labels that appear in the true or predicted values
    unique_labels = np.unique(np.concatenate([y_true, y_pred]))
    target_names_filtered = [class_names[i] for i in unique_labels]

    print(f"\nClassification Report for {name}")
    print(classification_report(y_true, y_pred, labels=unique_labels, target_names=target_names_filtered))

evaluate_model(custom_model, val_ds, "Custom CNN")
evaluate_model(mobilenet_model, val_ds, "MobileNetV2")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
